In [2]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn import preprocessing
import matplotlib.pyplot as plt

# Simple Neural network using MNIST data

## Download the dataset

In [4]:
mnist = keras.datasets.mnist

In [5]:
# this will load the dataset into train and test part automatically
(x_train, y_train),(x_test, y_test) = mnist.load_data()


In [6]:
# Normalizing the dataset
x_train = (x_train/255).astype('float32')
x_test = (x_test/255).astype('float32')

In [7]:
# if we check the labels we will see it is not one hot encoded yet.
y_train[0]

5

In [8]:
# using this library we will convert the labels into one hot encodings.
lb = preprocessing.LabelBinarizer()
y_train_hot = (lb.fit_transform(y_train)).astype('int32')
y_test_hot = (lb.fit_transform(y_test)).astype('int32')

In [10]:
# now if we check the labels we will see that it is one hot encoded.
y_train_hot[0]

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0], dtype=int32)

In [11]:
#spliting into train and validation sets
x_train, x_val, y_train_hot, y_val_hot = train_test_split(x_train, y_train_hot, test_size=0.33, random_state=42)

*mnist* data set object has methods to call next batches. For our better understanding, we will not use the premade method, instead, we will make a function to call the next batch. The following funtion does the job.

In [13]:
# as in batch calling we shuffle the data every time, that's why we did horizontal stacking so that the correct labels are not
# messed up.
def next_batch(x,y, batch_size):
    x = x.reshape(-1,28*28)    
    data = np.hstack((x,y))
    np.random.shuffle(data)
    x = data[:, :-10]
    y = data[:,-10:]
    x = x.reshape(-1,28,28,1)
    x = (x[:batch_size,:]).astype("float32")
    y = (y[:batch_size,:]).astype("int32")
    return x, y

In [14]:
batch_size = 100
n_batches = x_train.shape[0]/batch_size

In [15]:
#reshaping the data to feed into neural networks.
x_test = x_test.reshape(-1,28,28,1)
x_val = x_val.reshape(-1, 28,28,1)

So we defined the computational graph below. Its a very simple neural network with two hidden layers. The advantage of using mid level API is that it gives the flexibility to modify layers and at the same time, avoids the complication of defining each layers (weights, biases etc.) manually. The layers are defined using one line here. So the computational graph has one input layer which is a flatten layer. We need to flatten the data before feeding it to a dense layer. Hence, the flatten layer. Then we have two dense layers each followed by a relu activation. and last the output layer has a SOFTMAX activation with number of classes as the output size. You ask why SOFTMAX? we want the probability of each data to belonging to one of the 10 classes (ten numbers from 0 to 9).

In [16]:
"define the computational graph here"

tf.reset_default_graph()
x = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
y = tf.placeholder(tf.int32)
kernel_initializer = tf.initializers.truncated_normal(stddev=0.1)

flat = tf.layers.flatten(x)
l1 = tf.layers.dense(flat,128, kernel_initializer=kernel_initializer)
relu1 = tf.nn.relu(l1)
l2 = tf.layers.dense(relu1,32, kernel_initializer=kernel_initializer)
relu2 = tf.nn.relu(l2)

logits = tf.layers.dense(relu2, 10, kernel_initializer=kernel_initializer)
prob = tf.nn.softmax(logits)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prob,labels=y))
optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss)

In [18]:
# we will just run a few epochs to demonstrate the process.
num_ep = 2
with tf.Session() as sess:
    sess.run(tf.initializers.global_variables())
    
    for ep in range(num_ep):
        ep_loss = 0
        for steps in range(int(n_batches)):
            x_batch, y_batch_hot = next_batch(x_train,y_train_hot, batch_size)
            batch_loss, _ = sess.run([loss, training_op], feed_dict={x:x_batch,y:y_batch_hot})                                     
            ep_loss = ep_loss+batch_loss
            
        correct = tf.equal(tf.argmax(prob, 1), tf.argmax(y,1))
        accu = tf.reduce_mean(tf.cast(correct, tf.float32))
        accuracy = accu.eval({x:x_val, y:y_val_hot})
        print("epoch loss: {}".format(ep_loss))
        print("Accuracy after epoch {}: {}".format((ep+1),accuracy))
    
    # After the completed epochs we test the model    
    test_correct = tf.equal(tf.argmax(prob, 1), tf.argmax(y,1))
    test_accu = tf.reduce_mean(tf.cast(correct, tf.float32))
    test_accuracy = test_accu.eval({x:x_test, y:y_test_hot})
    print("Test Accuracy ", test_accuracy)

epoch loss: 672.3380879163742
Accuracy after epoch 1: 0.9196969866752625
epoch loss: 617.7031271457672
Accuracy after epoch 2: 0.9382323026657104
Test Accuracy  0.937
